# Scratch Work

In [1]:
#Create a df that has all the data for every time frame
#One row should be one timeframe, approx. one minute each

In [2]:
import numpy as np
import pandas as pd
import prepare
from time import sleep
import requests
from env import api_key

In [3]:
def get_match_info(match_ids, api_key, username, time = 20):
    """
    This function takes in a list of match ids and iterates through them. For each iteration,
    it will make two api calls and retrieve the necessary information for our project.
    
    Two json lists will be created, and once all the info has been gathered for each match id,
    this function will save both json lists for future reference. 
    
    This function also takes in a username string. This will be used for naming the saved files.
    
    Finally, this function will funnel the resulting json lists into a prepare function and
    return a complete df.
    
    The time parameter is used in the prepare function that will be called at the end
    and represents the timeframe of the data we want to gather and prepare.
    The default value is 20 for the 20 minute mark of the match.
    
    """
    
    #Create an empty list to store the timeline json data
    timeline_data = []
    
    #Create an empty list to store the other json data
    game_data = []
    
    #Set up main url
    url = 'https://americas.api.riotgames.com/'
    
    #Create a count var
    n = 0
    
    #Store the length of the match_ids list in a var
    match_count = len(match_ids)
    
    #Loop through each match_id
    for match in match_ids:
        #Update the count var
        n += 1
        
        #After every 50 match ids (100 total requests), wait 150 seconds (2.5 minutes)
        if n % 50 == 0:
            print(f'Completed {n} of {match_count} match IDs.')
            print(f'Waiting 150 seconds...\n')
            
            #Sleep 150 seconds
            sleep(150)
            
            print('Continuing!\n')
        
        ############################# TIMELINE DATA #######################
        #Set up timeline url
        timeline_query = f'lol/match/v5/matches/{match}/timeline/?api_key={api_key}'
        
        #Grab timeline json data
        timeline_response = requests.get(url + timeline_query)
        
        #Check response and leave a status message
        if timeline_response.status_code != 200:
            print(f'Something went wrong getting TIMELINE DATA! Status Code {timeline_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID.\n')
            continue

        #Turn it into json format
        timeline_json = timeline_response.json()
        
        #Append this data to the timeline_data list
        timeline_data.append(timeline_json)
        
        ############################# OTHER GAME DATA #######################
        #Set up game data url
        game_query = f'lol/match/v5/matches/{match}?api_key={api_key}'
        
        #Grab game json data
        game_response = requests.get(url + game_query)
        
        #Check response and leave a status message.
        if game_response.status_code != 200:
            print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')
            
            #Remove the last entry in the timeline_data list
            timeline_data.pop()
            continue
        
        #Turn it into json format
        game_json = game_response.json()
        
        #Append this data to the game_data list
        game_data.append(game_json)
        
    ################################## END LOOP ##############################
    
    #Now that all of the json data has been gathered, save each file for future reference
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)
    
    #Save as csv file. Use username string to identify whose file it is
    #timeline_df.to_json(f'timeline_data_{username}.json')
    
    #Print status message
    #print(f'Created timeline_data_{username}.json')
    
    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)
    
    #Save as csv file. Use username string to identify whose file it is
    #game_df.to_json(f'other_game_data_{username}.json')
    
    #Print status message
    #print(f'Created other_game_data_{username}.json\n')
    
    ################################ PREPARE DATA #########################
    
    #The following section will funnel the data into the prepare function
    #Written by Joshua C.
    
    #Leave a status message
    #print('Preparing the data...\n')
    
    
    #Finally, return the prepared df
    return timeline_data, game_data

In [8]:
#Set up match_id
match = ['NA1_4093908534', 'NA1_4094915650', 'NA1_4082128299', 'NA1_4083500908', 'NA1_4097058147']

#Set up username
username = 'smith'

#Get the match info
timeline_data, game_data = get_match_info(match, api_key, username)

In [9]:
len(timeline_data)

5

In [10]:
len(game_data)

5

In [11]:
#Loop through the prepare function 20 times for one match and concat all the resulting
#data frames into one

#Set up container df
match_info = pd.DataFrame()

#Get the data for one timeframe
df = prepare.prepare(timeline_data, game_data, 20)

#Now concat this info to match_info df
match_info.append(df, ignore_index = True)

Finished with: 0 of 5
Finished with: 1 of 5
Finished with: 2 of 5
Finished with: 3 of 5
Finished with: 4 of 5


,killsplayer_1,killsplayer_2,killsplayer_3,killsplayer_4,killsplayer_6,killsplayer_7,killsplayer_8,killsplayer_9,killsplayer_10,deathsplayer_1,...,team_totalGold_100,team_totalGold_200,team_trueDamageDoneToChampions_100,team_trueDamageDoneToChampions_200,team_ward_player_100,team_ward_player_200,team_assistsplayer_100,team_assistsplayer_200,team_xp_100,team_xp_200
0,5.0,1.0,2.0,2.0,3.0,5.0,9.0,3.0,1.0,4.0,...,34000,37365,2569,3457,60,99,7,24,38501,45035
1,1.0,6.0,7.0,1.0,0.0,4.0,2.0,3.0,1.0,2.0,...,33750,33208,1635,2169,165,100,14,15,41090,39724
2,2.0,0.0,10.0,6.0,2.0,3.0,2.0,6.0,1.0,2.0,...,38571,35469,1020,1899,127,344,27,18,45699,41224
3,3.0,9.0,5.0,3.0,3.0,4.0,7.0,1.0,0.0,5.0,...,39654,36342,2689,1590,53,58,20,19,42028,40431
4,1.0,2.0,0.0,2.0,0.0,3.0,6.0,2.0,1.0,3.0,...,31253,35913,1230,2203,64,71,6,12,40434,43535
